In [1]:
# RUN ONCE AND ONCE ONLY PER SESSION
import dask
from dask.distributed import Client, LocalCluster

try:
    client = Client('127.0.0.1:8786')
except:
    cluster = LocalCluster(scheduler_port=8786)
    client = Client(cluster)

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 31.75 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:8786,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 31.75 GiB
Comm: tcp://127.0.0.1:58775,Total threads: 3
Dashboard: http://127.0.0.1:58778/status,Memory: 7.94 GiB
Nanny: tcp://127.0.0.1:58752,


In [3]:
# Run as is
import pandas as pd
import numpy as np
import datetime
import glob, os
from collections import defaultdict
from multiprocessing import Pool, cpu_count
from functools import partial

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_cross_validation_metric
from fbprophet.make_holidays import make_holidays_df
from fbprophet.diagnostics import cross_validation, performance_metrics

from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, LabelEncoder

import plotly.express as px
import plotly.graph_objects as go

pd.set_option('display.max_columns', 50)
pd.set_option('display.precision', 2)
pd.options.display.float_format = '{:,.2f}'.format

C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Change the value in this bit of code "country_name = 'ZA'"
# This should reflect the country you are running the curve for
# Use the Alpha 3 code from this list: https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes

class ProphetPredictions:
    def __init__(self, model, figforecast, figcomponents, dfcrossvalidation, dfperformancemetrics, figcrossvalidation):
        self.model = model
        self.figforecast = figforecast
        self.figcomponents = figcomponents
        self.dfcrossvalidation = dfcrossvalidation
        self.dfperformancemetrics = dfperformancemetrics
        self.figcrossvalidation = figcrossvalidation

    def getmodel(self):
        return self.model


def mapefn(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def generate_prediction(myhols, dfpredict):
    #mdl = Prophet(seasonality_mode='multiplicative', seasonality_prior_scale=0.5)    
    ## add cap & floor for df & future
    mydf = dfpredict.copy()
    # Box-Cox transformation of Order 0
    mydf['y'] = np.log(1+mydf['y']) #log transform
    mydf['cap'] = mydf['y'].max()
    mydf['floor'] = 1
    
    mdl = Prophet(growth='logistic', changepoint_prior_scale=0.001, holidays=myhols, yearly_seasonality=True).fit(mydf, iter=100)
    
    # Create future dataset & forecast against
    future = mdl.make_future_dataframe(periods=52, freq='W')
    future['cap'] = mydf['y'].max()
    future['floor'] = 1
    #Maybe update hols here?

    # Predict
    forecast = mdl.predict(future)
    
    # Inverse Box-Cox
    forecast['yhat'] = np.exp(forecast['yhat']) - 1
    forecast['trend'] = np.exp(forecast['trend']) - 1

    # print("Calculating MAPE . . .")
    # df_mape = mapefn(dfpredict['y'], forecast['yhat'])

    return forecast  # ,df_mape,



# Use the Alpha 3 code from this list: https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes
def makeprophethols(timesdf, future_periods, date_freq):
    # Make Holidays with bounds
    year_list = [2019, 2020, 2021]
    country_name = 'FRA'
    myhols = make_holidays_df(year_list, country_name)

    # Create all dates
    myidx = timesdf.sort_values()
    myidx = myidx.drop_duplicates()
    firstdate = myidx[0]
    lastdate = myidx[-1]
    future_dates = pd.date_range(start=lastdate, periods=future_periods+1, freq=date_freq)
    future_dates = future_dates[future_dates > lastdate] # Drop start date if equal to lastdate
    myidx = myidx.append(future_dates[:future_periods]) # Return correct number of periods
    lastdate = myidx[-1] # Update last date
    
    # Select only holidays beginning from 7 days prior to start of dataset
    myhols = myhols.loc[myhols['ds'] > (firstdate - pd.offsets.Day(7))]

    # Select only holidays beginning from 7 days prior to start of dataset to ending last date of dataset
    myhols = myhols.loc[(myhols['ds'] > (firstdate - pd.offsets.Day(7))) & (myhols['ds'] <= lastdate)]
    myhols.reset_index(inplace=True, drop=True)

    # Finds the closest date for each holiday that can be found in your current dataset
    for i in range(len(myhols)):
        myhols.loc[i, 'refdate'] = myidx[myidx.get_loc(pd.to_datetime(myhols.loc[i, 'ds']), method='nearest')]


    myhols['datediff'] = myhols['refdate'] - myhols['ds']

    # Additional time delta ensures it runs up to end of month which is needed when forecasting future monthly data
    for i in range(len(myhols)):
        if myhols.loc[i, 'datediff'] >= datetime.timedelta(minutes=0):
            myhols.loc[i, 'upper_window'] = pd.to_timedelta(myhols.loc[i, 'refdate'] - myhols.loc[i, 'ds'])
        else:
            myhols.loc[i, 'upper_window'] = 0

    for i in range(len(myhols)):
        if myhols.loc[i, 'datediff'] < datetime.timedelta(minutes=0):
            myhols.loc[i, 'lower_window'] = pd.to_timedelta(myhols.loc[i, 'refdate'] - myhols.loc[i, 'ds'])
        else:
            myhols.loc[i, 'lower_window'] = 0

    myhols['upper_window'] = pd.to_timedelta(myhols['upper_window']).dt.days
    myhols['lower_window'] = pd.to_timedelta(myhols['lower_window']).dt.days
    myhols.drop(columns=['refdate', 'datediff'], inplace=True)
    # Set window for the future dataframe
    #To be fixed/created. Ignore future timeperiods for now
    return myhols

def generateweeklyaggs(df, valcol, distcol):
    df['val_pp1'] = df[valcol].shift(periods=1)
    df['dist_pp1'] = df[distcol].shift(periods=1)

    df['val_pp2'] = df[valcol].shift(periods=2)
    df['dist_pp2'] = df[distcol].shift(periods=2)

    df['val_pp3'] = df[valcol].shift(periods=3)
    df['dist_pp3'] = df[distcol].shift(periods=3)

    df['val_chg1'] = df[valcol].diff(periods=1)
    df['dist_chg1'] = df[distcol].diff(periods=1)

    df['val_chg2'] = df[valcol].diff(periods=2)
    df['dist_chg2'] = df[distcol].diff(periods=2)

    df['val_chg3'] = df[valcol].diff(periods=3)
    df['dist_chg3'] = df[distcol].diff(periods=3)

    df['val4wk'] = df[valcol].rolling(4).sum()
    df['dist4wk'] = df[distcol].rolling(4).sum()

    df['val_4wkchg1'] = df['val4wk'].diff(periods=1)
    df['dist_4wkchg1'] = df['dist4wk'].diff(periods=1)

    df['val_4wkchg2'] = df['val4wk'].diff(periods=2)
    df['dist_4wkchg2'] = df['dist4wk'].diff(periods=2)

    return df


def outlier_week_check(row, valchg, val_pp):
    ##Returns true if the % change vs. prev week is greater than 100%
    return (abs(row[valchg] / row[val_pp]) // 1) >= 1


def calculatesupplementarymetrics(df, valcol, distcol):
    df['val_chg1_perc'] = df['val_chg1'] / df['val_pp1']
    df['dist_chg1_perc'] = df['dist_chg1'] / df['dist_pp1']

    df['val_chg2_perc'] = df['val_chg2'] / df['val_pp3']
    df['dist_chg2_perc'] = df['dist_chg2'] / df['dist_pp3']

    df['val_4wkchg1_perc'] = df['val_4wkchg1'] / (df['val4wk'] + df['val_4wkchg1'])
    df['dist_4wkchg1_perc'] = df['dist_4wkchg1'] / (df['dist4wk'] + df['dist_4wkchg1'])

    df['val_4wkchg2_perc'] = df['val_4wkchg2'] / (
            df['val4wk'] + df['val_4wkchg1'] + df['val_4wkchg2'])
    df['dist_4wkchg2_perc'] = df['dist_4wkchg2'] / (
            df['dist4wk'] + df['dist_4wkchg1'] + df['dist_4wkchg2'])

    df.replace((np.inf, -np.inf), (0, 0), inplace=True)

    return df


def createfile(df_input, valcol, tdpcol):
    # Drop zero rows to avoid divide by zero errors
    df = df_input.copy()
    df.drop(df[df[valcol] == 0].index, inplace=True)
    df.drop(df[df[tdpcol] <= 0].index, inplace=True)

    df = generateweeklyaggs(df, valcol, tdpcol)
    df = calculatesupplementarymetrics(df, valcol, tdpcol)

    # Remove rows with less than 4weeks prior history
    df.drop(df[df['dist_chg1'].isna()].index, inplace=True)

    df['is_outlier(100%chg)'] = df.apply(outlier_week_check, axis=1, args=('val_chg1', 'val_pp1')).astype('int')
    return df

Change the name of the two csv files you will load in. First, the sales file/raw data and secondly, the segmentation file saved as a csv

In [5]:
my_path = r"C:\Users\ibrahimad\OneDrive - Kantar\Documents\Colgate\Colgate France\Curves 2022\TP\ISD\\"
files_list = glob.glob(my_path+"*.csv")
tt_raw_dict = {}
for f in files_list:
    temp_df = pd.DataFrame()
    temp_df = pd.read_csv(f, dtype={"Barcode":str})
    key_value = f.replace(my_path,"").replace(".csv","")
    temp_df['Region'] = key_value
    tt_raw_dict[key_value] = temp_df
    
#tt_raw_list = pd.concat([pd.read_csv(f, dtype={"Barcode":str}) for f in files_list])
tt_raw_dict.keys()

dict_keys(['C:\\Users\\ibrahimad\\OneDrive - Kantar\\Documents\\Colgate\\Colgate France\\Curves 2022\\TP\\ISD\\Curve File v2'])

In [6]:
tt_raw_orig = pd.concat([tt_raw_dict[f] for f in tt_raw_dict.keys()])
tt_raw_orig.describe()

,UNITS,DOLLAR,ACV
count,"249,929.00","249,929.00","249,929.00"
mean,525.61,"1,357.15",53.50
std,"1,032.83","2,274.08",38.51
min,0.50,0.10,1.00
25%,17.00,57.10,11.00
50%,129.00,434.80,60.00
75%,555.20,"1,594.80",93.00
max,"18,359.00","43,435.60",100.00


In [7]:
tt_raw = tt_raw_orig.loc[(tt_raw_orig['ACV'] <= 100) & (tt_raw_orig['ACV'] > 0) & (tt_raw_orig['UNITS'] > 0) & (tt_raw_orig['DOLLAR'] > 0)].reset_index(drop=True)
tt_raw.describe()

,UNITS,DOLLAR,ACV
count,"249,929.00","249,929.00","249,929.00"
mean,525.61,"1,357.15",53.50
std,"1,032.83","2,274.08",38.51
min,0.50,0.10,1.00
25%,17.00,57.10,11.00
50%,129.00,434.80,60.00
75%,555.20,"1,594.80",93.00
max,"18,359.00","43,435.60",100.00


In [8]:
tt_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249929 entries, 0 to 249928
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Retailer  249929 non-null  object 
 1   UPC_DESC  249929 non-null  object 
 2   WEEK      249929 non-null  object 
 3   UNITS     249929 non-null  float64
 4   DOLLAR    249929 non-null  float64
 5   ACV       249929 non-null  int64  
 6   Region    249929 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 13.3+ MB


In [9]:
tt_raw

,Retailer,UPC_DESC,WEEK,UNITS,DOLLAR,ACV,Region
0,Auchan,3283950920513BP1,2020-04-19,1.00,0.30,1,C:\Users\ibrahimad\OneDrive - Kantar\Documents...
1,Auchan,3283950920513BP1,2020-05-03,1.00,0.30,2,C:\Users\ibrahimad\OneDrive - Kantar\Documents...
2,Auchan,3283950920513BP1,2020-02-16,1.00,0.40,1,C:\Users\ibrahimad\OneDrive - Kantar\Documents...
3,Auchan,3283950920513BP1,2020-06-07,1.00,0.40,1,C:\Users\ibrahimad\OneDrive - Kantar\Documents...
4,Auchan,3700179430192,2020-05-10,1.00,0.60,2,C:\Users\ibrahimad\OneDrive - Kantar\Documents...
...,...,...,...,...,...,...,...
249924,U_enseigne,8718951284395,2020-03-01,"8,819.00","35,053.10",74,C:\Users\ibrahimad\OneDrive - Kantar\Documents...
249925,U_enseigne,8718951284395,2020-03-29,"9,017.00","35,923.20",74,C:\Users\ibrahimad\OneDrive - Kantar\Documents...
249926,U_enseigne,8718951284395,2020-03-08,"9,680.00","38,354.00",75,C:\Users\ibrahimad\OneDrive - Kantar\Documents...
249927,U_enseigne,8718951284395,2020-03-22,"10,652.80","42,312.60",75,C:\Users\ibrahimad\OneDrive - Kantar\Documents...


In [10]:
tt_raw['Retailer'].unique()

array(['Auchan    ', 'Auchan_SM ', 'Casino    ', 'Cora      ',
       'CRF_HM    ', 'CRF_Market', 'Geant     ', 'Monoprix  ',
       'U_enseigne'], dtype=object)

In [11]:
tt_raw['WEEK'].unique()

array(['2020-04-19', '2020-05-03', '2020-02-16', '2020-06-07',
       '2020-05-10', '2020-05-17', '2020-08-02', '2020-09-06',
       '2020-11-01', '2021-01-24', '2021-02-07', '2021-04-11',
       '2021-04-25', '2021-06-06', '2021-06-13', '2021-07-11',
       '2021-09-05', '2021-09-12', '2021-09-19', '2021-10-03',
       '2021-10-17', '2022-01-30', '2020-03-08', '2020-04-05',
       '2020-10-11', '2020-11-29', '2020-12-13', '2021-12-05',
       '2021-02-14', '2021-05-09', '2021-06-27', '2021-07-25',
       '2021-10-24', '2021-11-14', '2021-12-26', '2021-07-18',
       '2020-01-26', '2020-02-09', '2020-02-23', '2020-03-22',
       '2020-05-31', '2020-06-21', '2020-07-26', '2020-10-04',
       '2020-11-08', '2021-05-16', '2021-08-08', '2021-10-31',
       '2021-08-22', '2020-10-18', '2020-11-15', '2021-03-14',
       '2021-04-04', '2021-11-28', '2021-08-15', '2021-08-29',
       '2020-10-25', '2020-03-01', '2020-06-28', '2020-08-09',
       '2020-09-20', '2020-12-20', '2021-01-31', '2021-

In [12]:
tt_raw['WEEK'] = pd.to_datetime(tt_raw['WEEK'])
tt_raw['WEEK'].unique()

array(['2020-04-19T00:00:00.000000000', '2020-05-03T00:00:00.000000000',
       '2020-02-16T00:00:00.000000000', '2020-06-07T00:00:00.000000000',
       '2020-05-10T00:00:00.000000000', '2020-05-17T00:00:00.000000000',
       '2020-08-02T00:00:00.000000000', '2020-09-06T00:00:00.000000000',
       '2020-11-01T00:00:00.000000000', '2021-01-24T00:00:00.000000000',
       '2021-02-07T00:00:00.000000000', '2021-04-11T00:00:00.000000000',
       '2021-04-25T00:00:00.000000000', '2021-06-06T00:00:00.000000000',
       '2021-06-13T00:00:00.000000000', '2021-07-11T00:00:00.000000000',
       '2021-09-05T00:00:00.000000000', '2021-09-12T00:00:00.000000000',
       '2021-09-19T00:00:00.000000000', '2021-10-03T00:00:00.000000000',
       '2021-10-17T00:00:00.000000000', '2022-01-30T00:00:00.000000000',
       '2020-03-08T00:00:00.000000000', '2020-04-05T00:00:00.000000000',
       '2020-10-11T00:00:00.000000000', '2020-11-29T00:00:00.000000000',
       '2020-12-13T00:00:00.000000000', '2021-12-05

In [15]:
salesdf_raw = tt_raw[['Retailer','UPC_DESC','WEEK','UNITS','DOLLAR','ACV']].copy()
#salesdf_raw = tt.copy()
salesdf_raw

,Retailer,UPC_DESC,WEEK,UNITS,DOLLAR,ACV
0,Auchan,3283950920513BP1,2020-04-19,1.00,0.30,1
1,Auchan,3283950920513BP1,2020-05-03,1.00,0.30,2
2,Auchan,3283950920513BP1,2020-02-16,1.00,0.40,1
3,Auchan,3283950920513BP1,2020-06-07,1.00,0.40,1
4,Auchan,3700179430192,2020-05-10,1.00,0.60,2
...,...,...,...,...,...,...
249924,U_enseigne,8718951284395,2020-03-01,"8,819.00","35,053.10",74
249925,U_enseigne,8718951284395,2020-03-29,"9,017.00","35,923.20",74
249926,U_enseigne,8718951284395,2020-03-08,"9,680.00","38,354.00",75
249927,U_enseigne,8718951284395,2020-03-22,"10,652.80","42,312.60",75


In [25]:
# Load in your data file
#salesdf_raw = pd.read_csv('C:\\Users\\kuria\\Downloads\\Data Transform\\Asahi UK\\HANDS RAWDATA JAN 20.csv', header = 0, thousands = ',', error_bad_lines = True, warn_bad_lines = True)
segdf = pd.read_excel(my_path+'SEG_PL_Template_FR_CPFRTP_22.xlsx')
segdf

,UPC_DESC,CATEGORY,SEG_DESC,SUB_SEG_DESC,BASE_DESC,BASE_SIZE,MANUFACTURER,BRAND,US_UNITS,US_ACV,US_DOLLARS,PRODUCTDESCRIPTION,SEGMENT,BENEFICES,PACK,ProductAttribute4,ProductAttribute5,ProductAttribute6,ProductAttribute7,ProductAttribute8,ProductAttribute9,ProductAttribute10,ProductAttribute11,ProductAttribute12,ProductAttribute13,ProductAttribute14,ProductAttribute15,ProductAttribute16,ProductAttribute17,ProductAttribute18,ProductAttribute19,ProductAttribute20
0,3770018286334,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,AO MARQUES - ANTI CARIES,900 900.CARE MENTHE DOUCE BOITE 200 1X STD ADU...,1,900.CARE,900,NaN,2.00,NaN,900 900.CARE MENTHE DOUCE BOITE 200 1X STD ADU...,BIO NATURALITE,ANTI-CARIES,SOLIDE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3770018286341,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,AO MARQUES - ANTI CARIES,900 900.CARE MENTHE EXTRA FRAICHE BOITE 200 1X...,1,900.CARE,900,NaN,2.00,NaN,900 900.CARE MENTHE EXTRA FRAICHE BOITE 200 1X...,BIO NATURALITE,ANTI-CARIES,SOLIDE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3770018286655,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,AO MARQUES - ANTI CARIES,900 900.CARE MENTHE DOUCE BOITE 100 1X STD ADU...,1,900.CARE,900,NaN,1.00,NaN,900 900.CARE MENTHE DOUCE BOITE 100 1X STD ADU...,BIO NATURALITE,ANTI-CARIES,SOLIDE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8710447428320,Toothpaste (FR),ENFANTS,SIGNAL,SIGNAL KIDS 2-6 ANS FRAISE GAGA STAND UP 50 1X...,1,UNILEVER,SIGNAL,NaN,98.00,NaN,SIGNAL KIDS 2-6 ANS FRAISE GAGA STAND UP 50 1X...,ENFANTS,ENFANTS,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8717644091777,Toothpaste (FR),ENFANTS,SIGNAL,SIGNAL POKEMON 7+ANS MENTHE DOUCE TUB 75 1X ST...,1,UNILEVER,SIGNAL,NaN,98.00,NaN,SIGNAL POKEMON 7+ANS MENTHE DOUCE TUB 75 1X ST...,ENFANTS,ENFANTS,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,8710522605547,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,ZENDIUM,ZENDIUM FLUOR TUB 75 1X STD ADU PAT TUB * 8710...,1,UNILEVER,ZENDIUM,NaN,32.00,NaN,ZENDIUM FLUOR TUB 75 1X STD ADU PAT TUB *,THERAPEUTIQUE,FRAICHEUR,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461,8712561961196,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,ZENDIUM,ZENDIUM PROTECTION COMPLETE TUB 75 1X STD ADU ...,1,UNILEVER,ZENDIUM,NaN,4.00,NaN,ZENDIUM PROTECTION COMPLETE TUB 75 1X STD ADU ...,THERAPEUTIQUE,SOIN COMPLET,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,8710908311024,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,ZENDIUM,ZENDIUM BLANCHEUR ET DOUCEUR TUB 75 1X STD ADU...,1,UNILEVER,ZENDIUM,NaN,0.00,NaN,ZENDIUM BLANCHEUR ET DOUCEUR TUB 75 1X STD ADU...,THERAPEUTIQUE,BLANCHEUR,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
463,8710447263150,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,ZENDIUM,ZENDIUM EXTRA FESH TUB 75 1X STD ADU PAT TUB *...,1,UNILEVER,ZENDIUM,NaN,0.00,NaN,ZENDIUM EXTRA FESH TUB 75 1X STD ADU PAT TUB *,THERAPEUTIQUE,FRAICHEUR,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Run as is
segdf.columns

Index(['UPC_DESC', 'CATEGORY', 'SEG_DESC', 'SUB_SEG_DESC', 'BASE_DESC',
       'BASE_SIZE', 'MANUFACTURER', 'BRAND', 'US_UNITS', 'US_ACV',
       'US_DOLLARS', 'PRODUCTDESCRIPTION', 'SEGMENT', 'BENEFICES', 'PACK',
       'ProductAttribute4', 'ProductAttribute5', 'ProductAttribute6',
       'ProductAttribute7', 'ProductAttribute8', 'ProductAttribute9',
       'ProductAttribute10', 'ProductAttribute11', 'ProductAttribute12',
       'ProductAttribute13', 'ProductAttribute14', 'ProductAttribute15',
       'ProductAttribute16', 'ProductAttribute17', 'ProductAttribute18',
       'ProductAttribute19', 'ProductAttribute20'],
      dtype='object')

Edit the left hand side of the colon which should match the list of columns above

Ensure you replace "Barcode" below with the name of your UPC column as per above and so on

In [27]:
# optional
segdf.rename(columns={'UPC_DESC': 'UPC',
                     }, inplace=True)

Ensure the output of the table below:

1) Has data that is in line with your expectations e.g. Mean & Standard Deviation

2) Has combined both raw sales data & segmentation data files

In [28]:
salesdf_raw.describe()

,UNITS,DOLLAR,ACV
count,"249,929.00","249,929.00","249,929.00"
mean,525.61,"1,357.15",53.50
std,"1,032.83","2,274.08",38.51
min,0.50,0.10,1.00
25%,17.00,57.10,11.00
50%,129.00,434.80,60.00
75%,555.20,"1,594.80",93.00
max,"18,359.00","43,435.60",100.00


If your file has multiple values OR Units columns e.g. Base & Total, drop the type you don't need

The code below drops the Total type as the base column has sufficient rows & correct info to use instead of the totals

Replace UNITS, DOLLARS with the name of the columns you will drop

In [29]:
# Run as is
salesdf_raw.columns

Index(['Retailer', 'UPC_DESC', 'WEEK', 'UNITS', 'DOLLAR', 'ACV'], dtype='object')

In [30]:
# If you do have extra columns to drop, uncomment the line below and run
#salesdf_raw.drop(columns=['Product_Description','MARKET'], inplace=True)
#salesdf_raw.drop(columns=['WEEK'], inplace=True)

Ensure you replace "BASE_UNITS" with the name of your Units column as per above and so on where needed
Only edit the left hand side of the colon which should match the list of columns above

In [31]:
salesdf = salesdf_raw.rename(columns={'WEEK':'ds', 
                                      'UPC_DESC': 'UPC',
                                      'Retailer':'Retailer',
                                      'UNITS':'UNITS',
                                      'DOLLAR':'VALUE',
                                      'ACV':'ACV'
                                     }
                            )

# Ensures matches will work as expected via string matching
salesdf['UPC'] = salesdf['UPC'].astype(str)
segdf['UPC'] = segdf['UPC'].astype(str)

salesdf = salesdf.merge(segdf, on='UPC')
# Find all non-UPC seg columns and rename them
for col in salesdf.columns[salesdf.columns.isin(segdf.columns[~segdf.columns.str.contains('UPC')])]:
    salesdf.rename(columns = {col: col+'_ATTRIBUTE'}, inplace=True)
    
salesdf

,Retailer,UPC,ds,UNITS,VALUE,ACV,CATEGORY_ATTRIBUTE,SEG_DESC_ATTRIBUTE,SUB_SEG_DESC_ATTRIBUTE,BASE_DESC_ATTRIBUTE,BASE_SIZE_ATTRIBUTE,MANUFACTURER_ATTRIBUTE,BRAND_ATTRIBUTE,US_UNITS_ATTRIBUTE,US_ACV_ATTRIBUTE,US_DOLLARS_ATTRIBUTE,PRODUCTDESCRIPTION_ATTRIBUTE,SEGMENT_ATTRIBUTE,BENEFICES_ATTRIBUTE,PACK_ATTRIBUTE,ProductAttribute4_ATTRIBUTE,ProductAttribute5_ATTRIBUTE,ProductAttribute6_ATTRIBUTE,ProductAttribute7_ATTRIBUTE,ProductAttribute8_ATTRIBUTE,ProductAttribute9_ATTRIBUTE,ProductAttribute10_ATTRIBUTE,ProductAttribute11_ATTRIBUTE,ProductAttribute12_ATTRIBUTE,ProductAttribute13_ATTRIBUTE,ProductAttribute14_ATTRIBUTE,ProductAttribute15_ATTRIBUTE,ProductAttribute16_ATTRIBUTE,ProductAttribute17_ATTRIBUTE,ProductAttribute18_ATTRIBUTE,ProductAttribute19_ATTRIBUTE,ProductAttribute20_ATTRIBUTE
0,Auchan,8714789357935,2021-07-18,1.00,0.70,1,Toothpaste (FR),ADULTE - NON PHARMA - COLGATE,BLANCHEUR,COLGATE MAX WHITE CRISTAUX ET BLANCHEUR TUB 75...,1,CP,COLGATE,NaN,2.00,NaN,COLGATE MAX WHITE CRISTAUX ET BLANCHEUR TUB 75...,GENERALISTES CLASSIQUES,BLANCHEUR,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Auchan,8714789357935,2021-01-17,1.00,1.90,1,Toothpaste (FR),ADULTE - NON PHARMA - COLGATE,BLANCHEUR,COLGATE MAX WHITE CRISTAUX ET BLANCHEUR TUB 75...,1,CP,COLGATE,NaN,2.00,NaN,COLGATE MAX WHITE CRISTAUX ET BLANCHEUR TUB 75...,GENERALISTES CLASSIQUES,BLANCHEUR,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Auchan,8714789357935,2021-02-14,1.00,1.90,1,Toothpaste (FR),ADULTE - NON PHARMA - COLGATE,BLANCHEUR,COLGATE MAX WHITE CRISTAUX ET BLANCHEUR TUB 75...,1,CP,COLGATE,NaN,2.00,NaN,COLGATE MAX WHITE CRISTAUX ET BLANCHEUR TUB 75...,GENERALISTES CLASSIQUES,BLANCHEUR,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Auchan,8714789357935,2020-11-08,3.00,5.80,2,Toothpaste (FR),ADULTE - NON PHARMA - COLGATE,BLANCHEUR,COLGATE MAX WHITE CRISTAUX ET BLANCHEUR TUB 75...,1,CP,COLGATE,NaN,2.00,NaN,COLGATE MAX WHITE CRISTAUX ET BLANCHEUR TUB 75...,GENERALISTES CLASSIQUES,BLANCHEUR,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Auchan,8714789357935,2020-10-11,3.00,6.00,2,Toothpaste (FR),ADULTE - NON PHARMA - COLGATE,BLANCHEUR,COLGATE MAX WHITE CRISTAUX ET BLANCHEUR TUB 75...,1,CP,COLGATE,NaN,2.00,NaN,COLGATE MAX WHITE CRISTAUX ET BLANCHEUR TUB 75...,GENERALISTES CLASSIQUES,BLANCHEUR,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234712,U_enseigne,3256228557278,2021-08-29,"1,044.20","2,087.10",81,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,AO MARQUES - BIO GREEN,DIST SOUS MARQUE DIST MENTHE & THE VERT BIO TU...,1,DIS,DIST,NaN,9.00,NaN,DIST SOUS MARQUE DIST MENTHE & THE VERT BIO TU...,MDD,BIO GREEN,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234713,U_enseigne,3256228557278,2021-10-31,"1,041.00","2,093.00",86,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,AO MARQUES - BIO GREEN,DIST SOUS MARQUE DIST MENTHE & THE VERT BIO TU...,1,DIS,DIST,NaN,9.00,NaN,DIST SOUS MARQUE DIST MENTHE & THE VERT BIO TU...,MDD,BIO GREEN,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234714,U_enseigne,3256228557278,2021-08-01,"1,074.10","2,143.90",83,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,AO MARQUES - BIO GREEN,DIST SOUS MARQUE DIST MENTHE & THE VERT BIO TU...,1,DIS,DIST,NaN,9.00,NaN,DIST SOUS MARQUE DIST MENTHE & THE VERT BIO TU...,MDD,BIO GREEN,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234715,U_enseigne,3256228557278,2022-01-09,"1,147.90","2,308.00",84,Toothpaste (FR),ADULTE - NON PHARMA - AO MARQUES,AO MARQUES - BIO GREEN,DIST SOUS MARQUE DIST MENTHE & THE VERT BIO TU...,1,DIS,DIST,NaN,9.00,NaN,DIST SOUS MARQUE DIST MENTHE & THE VERT BIO TU...,MDD,BIO GREEN,TUBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

Fill in the two variables below (ANALYSIS_CATEGORY & FOCUS_ATTRIBUTE_COLUMN) with the appropriate value

In [32]:
salesdf.columns

Index(['Retailer', 'UPC', 'ds', 'UNITS', 'VALUE', 'ACV', 'CATEGORY_ATTRIBUTE',
       'SEG_DESC_ATTRIBUTE', 'SUB_SEG_DESC_ATTRIBUTE', 'BASE_DESC_ATTRIBUTE',
       'BASE_SIZE_ATTRIBUTE', 'MANUFACTURER_ATTRIBUTE', 'BRAND_ATTRIBUTE',
       'US_UNITS_ATTRIBUTE', 'US_ACV_ATTRIBUTE', 'US_DOLLARS_ATTRIBUTE',
       'PRODUCTDESCRIPTION_ATTRIBUTE', 'SEGMENT_ATTRIBUTE',
       'BENEFICES_ATTRIBUTE', 'PACK_ATTRIBUTE', 'ProductAttribute4_ATTRIBUTE',
       'ProductAttribute5_ATTRIBUTE', 'ProductAttribute6_ATTRIBUTE',
       'ProductAttribute7_ATTRIBUTE', 'ProductAttribute8_ATTRIBUTE',
       'ProductAttribute9_ATTRIBUTE', 'ProductAttribute10_ATTRIBUTE',
       'ProductAttribute11_ATTRIBUTE', 'ProductAttribute12_ATTRIBUTE',
       'ProductAttribute13_ATTRIBUTE', 'ProductAttribute14_ATTRIBUTE',
       'ProductAttribute15_ATTRIBUTE', 'ProductAttribute16_ATTRIBUTE',
       'ProductAttribute17_ATTRIBUTE', 'ProductAttribute18_ATTRIBUTE',
       'ProductAttribute19_ATTRIBUTE', 'ProductAttribute20_ATTRI

In [33]:
# This is the name of your category & name-prefix of the output files
ANALYSIS_CATEGORY = 'CPFR_TP_ISD'
# This is the attribute you are creating an attribute specific curve for
# If you're running ISD, it is usually SEG_DESC_ATTRIBUTE
FOCUS_ATTRIBUTE_COLUMN = 'SEG_DESC_ATTRIBUTE'

print(f"There are {salesdf[FOCUS_ATTRIBUTE_COLUMN].nunique()} unique values in {FOCUS_ATTRIBUTE_COLUMN}\n The relative shares are as below:")
salesdf[[FOCUS_ATTRIBUTE_COLUMN, 'VALUE']].groupby(FOCUS_ATTRIBUTE_COLUMN).sum()/salesdf['VALUE'].sum()

There are 10 unique values in SEG_DESC_ATTRIBUTE
 The relative shares are as below:


,VALUE
SEG_DESC_ATTRIBUTE,
ADULTE - NON PHARMA - AO MARQUES,0.12
ADULTE - NON PHARMA - COLGATE,0.08
ADULTE - NON PHARMA - ORAL B,0.08
ADULTE - NON PHARMA - SIGNAL,0.28
ADULTE - NON PHARMA - VADMECUM,0.04
ADULTE - PHARMA - ELMEX,0.10
ADULTE - PHARMA - PARADONTAX,0.05
ADULTE - PHARMA - SANOGYL,0.03
ADULTE - PHARMA - SENSODYNE,0.17


In [34]:
# Run as is
# This creates the working dataframe & deletes rows with NA values
forecastdf = salesdf[[FOCUS_ATTRIBUTE_COLUMN,'Retailer','ds','UNITS','VALUE','ACV']].groupby([FOCUS_ATTRIBUTE_COLUMN,'Retailer','ds']).sum().rename(
    columns={'UNITS':'y'}).reset_index()

prev_row_cnt = forecastdf.shape[0]
forecastdf.dropna(subset=['y'], inplace=True) # Drop NAs
forecastdf = forecastdf.loc[forecastdf['y'] > 0] # Select strictly positive, non-zero values

"""
# Delete attribute-markets with low support = less than 50 rows by market & attribute 
# This is fine to ignore for now as e.g. Herbs will have representation in other markets
rowcounts = forecastdf.groupby([FOCUS_ATTRIBUTE_COLUMN,'MARKET']).count().reset_index()
rows_to_delist = rowcounts.loc[rowcounts['y'] < 52,[FOCUS_ATTRIBUTE_COLUMN,'MARKET']]
# Convert to str then join & select rows not matching rows_to_delist
forecastdf = forecastdf.loc[~(forecastdf[FOCUS_ATTRIBUTE_COLUMN].astype(str) + forecastdf['MARKET'].astype(str)).isin(
    rows_to_delist[FOCUS_ATTRIBUTE_COLUMN].astype(str) + rows_to_delist['MARKET'].astype(str))]
"""
current_row_cnt = forecastdf.shape[0]

print(f'Dropped {current_row_cnt - prev_row_cnt} rows with NA values. Final dataset has {current_row_cnt} rows')

Dropped 0 rows with NA values. Final dataset has 9665 rows


In [35]:
# Run as is
# Summary of the data rows by market & unique time
attr_sparsity_report = forecastdf[['Retailer','ds',FOCUS_ATTRIBUTE_COLUMN]].groupby(['Retailer',FOCUS_ATTRIBUTE_COLUMN]).count().reset_index().pivot(
    index='Retailer', columns=FOCUS_ATTRIBUTE_COLUMN, values='ds')

num_unique_timeperiods = forecastdf['ds'].nunique()
for col in attr_sparsity_report.columns:
    attr_sparsity_report[col] = attr_sparsity_report[col]/num_unique_timeperiods

#attr_sparsity_report.to_excel('./attr_sparsity_report_'+FOCUS_ATTRIBUTE_COLUMN+'.xlsx')
attr_sparsity_report.describe()

SEG_DESC_ATTRIBUTE,ADULTE - NON PHARMA - AO MARQUES,ADULTE - NON PHARMA - COLGATE,ADULTE - NON PHARMA - ORAL B,ADULTE - NON PHARMA - SIGNAL,ADULTE - NON PHARMA - VADMECUM,ADULTE - PHARMA - ELMEX,ADULTE - PHARMA - PARADONTAX,ADULTE - PHARMA - SANOGYL,ADULTE - PHARMA - SENSODYNE,ENFANTS
count,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00
mean,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.95,1.00,1.00
std,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.14,0.00,0.00
min,1.00,1.00,0.97,1.00,0.99,1.00,1.00,0.56,0.99,1.00
25%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
50%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
75%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [36]:
# Run as is
prev_row_cnt = forecastdf.shape[0]

tt = forecastdf[['Retailer','ds',FOCUS_ATTRIBUTE_COLUMN]].groupby(['Retailer',FOCUS_ATTRIBUTE_COLUMN]).count().reset_index()
tt.loc[:,'share_of_total_ds_with_data'] = tt.loc[:,'ds']/num_unique_timeperiods
drop_threshold = 0.5 # This is equivalent to 1yr if input is 2yrs worth of data. If only 1yr of data then set to 1
rows_to_keep = (tt.loc[tt['share_of_total_ds_with_data'] >= drop_threshold, 'Retailer'].str.cat(tt.loc[tt['share_of_total_ds_with_data'] >= drop_threshold, FOCUS_ATTRIBUTE_COLUMN])).tolist() # Drop rows will be those with < 1 yr of data to forecast with + nas
forecastdf = forecastdf.loc[forecastdf['Retailer'].str.cat(forecastdf[FOCUS_ATTRIBUTE_COLUMN]).isin(rows_to_keep)]

current_row_cnt = forecastdf.shape[0]

print(f'Dropped {current_row_cnt - prev_row_cnt} rows whose markets/attributes had insufficient data for forecasting. \nFinal dataset has {current_row_cnt} rows')



Dropped 0 rows whose markets/attributes had insufficient data for forecasting. 
Final dataset has 9665 rows


In [37]:
# Run as is
# Determine attributes to merge
"""
# Grouping Approach A -  Count of time period approach
# Calculate the % of Markets where the attribute is NaN i.e. Doesn't exist
count_sparsity = pd.DataFrame(attr_sparsity_report.count()/attr_sparsity_report.count().max())
mean_timeperiods_sparsity = pd.DataFrame(attr_sparsity_report.mean())
merge_threshold = 0.5 #40%

# Merge any attributes that are missing data in less than merge_threshold market/time_periods
attributes_to_group = count_sparsity.loc[count_sparsity[0] < merge_threshold].merge(
    mean_timeperiods_sparsity.loc[mean_timeperiods_sparsity[0] < merge_threshold], 
    on=FOCUS_ATTRIBUTE_COLUMN, how='outer').index.to_list()
"""
# Grouping Approach B -  Share of y
sparsity_y_based = forecastdf[[FOCUS_ATTRIBUTE_COLUMN,'y']].groupby([FOCUS_ATTRIBUTE_COLUMN]).agg(y_sum=("y","sum")).reset_index()
sparsity_y_based['y_share'] = sparsity_y_based['y_sum']/sparsity_y_based['y_sum'].sum()*100

try:
    attributes_to_group = sparsity_y_based.loc[sparsity_y_based['y_share'] < 0.05, FOCUS_ATTRIBUTE_COLUMN].unique().to_list()
    print(f"Merging the attributes below into an ALL OTHER group due to their low support: \n\n {attributes_to_group}")
except AttributeError: #empty list returned
    attributes_to_group = []
    print("There were no attributes to merge")
                                           

There were no attributes to merge


In [38]:
# Run as is
# Groups attributes if they have been flagged for merging
grouped_attributes_forecastdf = forecastdf.loc[forecastdf[FOCUS_ATTRIBUTE_COLUMN].isin(attributes_to_group)].groupby(['Retailer','ds']).sum().dropna(how='all').reset_index()
grouped_attributes_forecastdf[FOCUS_ATTRIBUTE_COLUMN] = 'ALL_OTHER_'+FOCUS_ATTRIBUTE_COLUMN

# Drop all rows which have been grouped then concatenate with the new grouped rows
forecastdf = pd.concat([forecastdf.loc[~forecastdf[FOCUS_ATTRIBUTE_COLUMN].isin(attributes_to_group)].reset_index(drop=True), grouped_attributes_forecastdf], 
          axis=0, ignore_index=True, sort=False).sort_values(by=['Retailer',FOCUS_ATTRIBUTE_COLUMN,'ds']).reset_index(drop=True)

# Summary of the data rows by market & unique time
attr_sparsity_report = forecastdf[['Retailer','ds',FOCUS_ATTRIBUTE_COLUMN]].groupby(['Retailer',FOCUS_ATTRIBUTE_COLUMN]).count().reset_index().pivot(
    index='Retailer', columns=FOCUS_ATTRIBUTE_COLUMN, values='ds')

num_unique_timeperiods = forecastdf['ds'].nunique()
for col in attr_sparsity_report.columns:
    attr_sparsity_report[col] = attr_sparsity_report[col]/num_unique_timeperiods

#attr_sparsity_report.to_excel('./attr_sparsity_report_'+FOCUS_ATTRIBUTE_COLUMN+'.xlsx')
attr_sparsity_report.describe()

SEG_DESC_ATTRIBUTE,ADULTE - NON PHARMA - AO MARQUES,ADULTE - NON PHARMA - COLGATE,ADULTE - NON PHARMA - ORAL B,ADULTE - NON PHARMA - SIGNAL,ADULTE - NON PHARMA - VADMECUM,ADULTE - PHARMA - ELMEX,ADULTE - PHARMA - PARADONTAX,ADULTE - PHARMA - SANOGYL,ADULTE - PHARMA - SENSODYNE,ENFANTS
count,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00,9.00
mean,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.95,1.00,1.00
std,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.14,0.00,0.00
min,1.00,1.00,0.97,1.00,0.99,1.00,1.00,0.56,0.99,1.00
25%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
50%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
75%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [39]:
forecastdf = forecastdf.loc[forecastdf['y']>0]

This creates the forecasts by attribute value & market that we will use to factor out other external changes

This code takes a while depending on the factors above. Typical is 3-5 mins if you have a "reasonable" # of unique values ~ < 30

May take upwards of 30 mins for high cardinality attributes. I recommend clustering these before importing

In [40]:
@dask.delayed
def generate_prediction_delayed(dfpredict, myhols):
    #mdl = Prophet(seasonality_mode='multiplicative', seasonality_prior_scale=0.5)    
    ## add cap & floor for df & future
    mydf = dfpredict.copy()
    # Box-Cox transformation of Order 0
    mydf['y'] = np.log(1+mydf['y']) #log transform
    mydf['cap'] = mydf['y'].max()
    mydf['floor'] = 1
    
    mdl = Prophet(growth='logistic', changepoint_prior_scale=0.001, holidays=myhols, yearly_seasonality=True).fit(mydf, iter=100)
    
    # Create future dataset & forecast against
    future = mdl.make_future_dataframe(periods=52, freq='W')
    future['cap'] = mydf['y'].max()
    future['floor'] = 1
    #Maybe update hols here?

    # Predict
    forecast = mdl.predict(future)
    
    # Inverse Box-Cox
    forecast['yhat'] = np.exp(forecast['yhat']) - 1
    forecast['trend'] = np.exp(forecast['trend']) - 1

    # print("Calculating MAPE . . .")
    # df_mape = mapefn(dfpredict['y'], forecast['yhat'])

    # return mdl, df_mape
    return dask.delayed(forecast) # return delayed object since the function is expected to return a delayed
    


In [41]:
# Run as is
# Create dict of dfs which will each be individually forecasted by market & attribute
dictdfs = {k: v for k, v in forecastdf.groupby([FOCUS_ATTRIBUTE_COLUMN,'Retailer'])}
times_list = pd.to_datetime(forecastdf['ds'].to_list())
myhols = makeprophethols(times_list, future_periods=52, date_freq='W')
predictiondict = defaultdict(Prophet)
forecastdict = defaultdict(pd.DataFrame)

C:\Users\ibrahimad\AppData\Local\Temp\ipykernel_11984\2276330627.py:80: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  myhols.loc[i, 'refdate'] = myidx[myidx.get_loc(pd.to_datetime(myhols.loc[i, 'ds']), method='nearest')]
C:\Users\ibrahimad\AppData\Local\Temp\ipykernel_11984\2276330627.py:80: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  myhols.loc[i, 'refdate'] = myidx[myidx.get_loc(pd.to_datetime(myhols.loc[i, 'ds']), method='nearest')]


In [42]:
# Run as is

forecast_holder = []


for df_key, dfpredict in dictdfs.items():
    dfpredict_delayed = dask.delayed(dfpredict)
    forecast_holder.append(generate_prediction_delayed(dfpredict_delayed, myhols))
    #forecast_holder['MARKET'] = df_key
    #print(f"Type: {type(forecastdict[df_key])}")

#forecast_holder.append(forecastdf.groupby([FOCUS_ATTRIBUTE_COLUMN,'MARKET']).apply(generate_prediction_delayed(dfpredict_delayed, myhols)))
# This works but I don't understand why I have to compute twice (here and cell below) given how costly the function is
forecasts = dask.delayed(forecast_holder).compute()
forecasts

[Delayed('DataFrame-2ab32fc2-bd59-4432-b811-6d780f87f49d'),
 Delayed('DataFrame-392a9f01-f9a8-402e-96e8-2bddb50cd3a6'),
 Delayed('DataFrame-52bd3fd2-bf8a-4ddf-8d5a-122cfa3ae0b7'),
 Delayed('DataFrame-e2f0c65b-3c0c-4c41-adda-ff251f4c8461'),
 Delayed('DataFrame-76c65a8f-9151-49f7-9bef-099c95407077'),
 Delayed('DataFrame-65dfc518-9c41-44e3-a943-cab2a550b658'),
 Delayed('DataFrame-5608032a-059c-4711-aa0a-59f630c43e92'),
 Delayed('DataFrame-0ac591b6-d2b1-44d6-9e71-eebd4c8c3e71'),
 Delayed('DataFrame-da231611-81f2-4a5d-8c02-44820aaa4394'),
 Delayed('DataFrame-85dce503-a94d-4c0d-9f1d-37bbe2bac0ab'),
 Delayed('DataFrame-bfa18d5b-0cbc-4b40-a069-53a27be05843'),
 Delayed('DataFrame-fb13b122-2d80-4596-9b39-6f8d8429a9eb'),
 Delayed('DataFrame-0c1f4406-dd2a-4360-b243-169902ee450c'),
 Delayed('DataFrame-5b895764-c8fc-4432-8f67-93aa11a155f4'),
 Delayed('DataFrame-dbb96252-af07-40ea-802b-33bb84e8009b'),
 Delayed('DataFrame-c72028f3-0bff-41ea-9a0b-e22908e93143'),
 Delayed('DataFrame-aaf74f4c-aa75-4c91-b

In [43]:
# Run as is
# y = the metric of focus -> typically should be BASE_UNITS
# yhat = forecasted value for y in that attribute, period & market
# catsales_nonforecast = y - yhat

dfcatforecast = pd.DataFrame()
for i in range(len(dictdfs.keys())):
    tempdf = forecasts[i].compute()
    tempdf[FOCUS_ATTRIBUTE_COLUMN], tempdf['Retailer'] = list(dictdfs.keys())[i]
    dfcatforecast = dfcatforecast.append(tempdf)
    
forecastdf = forecastdf.merge(dfcatforecast[[FOCUS_ATTRIBUTE_COLUMN,'Retailer','ds','yhat','trend']], 
                              how='left', on=[FOCUS_ATTRIBUTE_COLUMN,'Retailer','ds'], suffixes=("","_y"))
#forecastdf['catsales_nonforecast'] = forecastdf['y'] - forecastdf['yhat']
forecastdf['catsales_nonforecast'] = forecastdf['y'] - forecastdf['trend']

forecastdf[['y','yhat','trend','catsales_nonforecast']].describe()

C:\Users\ibrahimad\AppData\Local\Temp\ipykernel_11984\2649080786.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcatforecast = dfcatforecast.append(tempdf)
C:\Users\ibrahimad\AppData\Local\Temp\ipykernel_11984\2649080786.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcatforecast = dfcatforecast.append(tempdf)
C:\Users\ibrahimad\AppData\Local\Temp\ipykernel_11984\2649080786.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcatforecast = dfcatforecast.append(tempdf)
C:\Users\ibrahimad\AppData\Local\Temp\ipykernel_11984\2649080786.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfcatforecast = dfcatforecast.append(

,y,yhat,trend,catsales_nonforecast
count,"9,665.00","9,665.00","9,665.00","9,665.00"
mean,"13,520.20","13,667.48","13,296.33",223.87
std,"18,881.35","17,541.24","17,079.22","5,022.83"
min,1.00,1.58,1.74,"-27,738.45"
25%,"3,005.90","3,211.76","3,254.05",-953.28
50%,"7,061.90","7,141.09","7,179.82",-117.10
75%,"16,832.70","17,312.64","17,009.78",500.64
max,"151,339.30","116,412.16","107,385.01","87,045.48"


In [44]:
# Run as is
forecastdf_filtered = forecastdf.copy()
forecastdf_filtered['catsales_nonforecast_perc'] = forecastdf_filtered['catsales_nonforecast']/forecastdf_filtered['y']
# If deviation is <1%, set to np.nan
forecastdf_filtered.loc[abs(forecastdf_filtered['catsales_nonforecast_perc']) < 0.01, 'catsales_nonforecast'] = np.nan 
forecastdf_filtered.drop('catsales_nonforecast_perc', axis=1, inplace=True)
# Check for % of sales that are not accounted for by forecasted trend & seasonality
pd.DataFrame(forecastdf_filtered['catsales_nonforecast']/forecastdf_filtered['y']).describe()


,0
count,"9,142.00"
mean,-5.91
std,272.18
min,"-17,383.25"
25%,-0.16
50%,-0.04
75%,0.09
max,0.91


In [45]:
# Run as is
# Merge category forecast to original dataset & create regression base dataset
#forecastdf_new = createfile(forecastdf_filtered, valcol='catsales_nonforecast', tdpcol='ACV').dropna(how='all')
forecastdf_new = createfile(forecastdf_filtered, valcol='y', tdpcol='ACV').dropna(how='all')
# "Saves" value of focus attribute in a new column as this will be removed by get_dummies
forecastdf_new.loc[:,FOCUS_ATTRIBUTE_COLUMN+'2'] = forecastdf_new.loc[:,FOCUS_ATTRIBUTE_COLUMN].copy()

metriccols= forecastdf_new.columns[forecastdf_new.columns.str.contains('perc')].to_flat_index().to_list()
dummycols = list(FOCUS_ATTRIBUTE_COLUMN + "_"+ forecastdf_new[FOCUS_ATTRIBUTE_COLUMN].unique())
forecastdf_dummy = pd.get_dummies(forecastdf_new, columns=[FOCUS_ATTRIBUTE_COLUMN])
for col in dummycols:
    for col2 in metriccols:
        forecastdf_dummy[col2 +'_' +col] = forecastdf_dummy[col] * forecastdf_dummy[col2]
        
# Restores focus attribute column
forecastdf_dummy.rename(columns={FOCUS_ATTRIBUTE_COLUMN+'2':FOCUS_ATTRIBUTE_COLUMN}, inplace=True)

# Create regression aggregated dataset - By Market & Time Period
forecastdf_dummy.replace(0, np.nan, inplace=True) #Prevent zeros from messing the means
forecastdf_dummy_agg = forecastdf_dummy.loc[:,
                                            [FOCUS_ATTRIBUTE_COLUMN,'Retailer','ds'] + 
                                            forecastdf_dummy.columns[
                                                forecastdf_dummy.columns.str.contains('perc') & 
                                                # ensure line below has your attribute name
                                                forecastdf_dummy.columns.str.contains(FOCUS_ATTRIBUTE_COLUMN) &
                                                ~forecastdf_dummy.columns.str.contains('4wk')
                                            ].to_flat_index().to_list()].groupby(['Retailer','ds']).mean().reset_index()


forecastdf_dummy_agg.describe()

,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ENFANTS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ENFANTS,val_chg2_perc_SEG_DESC_ATTRIBUTE_ENFANTS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ENFANTS
count,970.00,956.00,969.00,950.00,972.00,950.00,971.00,958.00,969.00,956.00,969.00,960.00,972.00,961.00,972.00,962.00,971.00,895.00,970.00,911.00,971.00,909.00,971.00,932.00,972.00,834.00,972.00,869.00,918.00,841.00,918.00,852.00,970.00,936.00,971.00,946.00,972.00,947.00,972.00,943.00
mean,0.04,0.01,0.07,0.02,1.30,0.38,0.09,-0.01,2.80,2.86,0.50,-0.04,0.23,0.01,0.17,0.01,0.77,0.03,0.33,0.01,0.04,0.01,0.03,0.01,0.01,0.00,0.01,0.00,2.15,1.17,3.67,-0.00,42.87,2.64,31.12,3.66,0.01,-0.00,0.01,-0.00
std,0.67,0.20,0.95,0.28,29.08,8.17,1.29,0.15,79.05,88.10,12.38,1.43,2.35,0.10,1.53,0.12,9.56,0.44,4.70,0.39,0.47,0.13,0.37,0.09,0.16,0.09,0.24,0.12,49.05,29.75,110.33,0.16,806.23,52.02,634.17,65.17,0.21,0.07,0.23,0.09
min,-0.44,-0.33,-2.13,-0.32,-0.99,-1.00,-7.97,-2.40,-1.00,-1.00,-13.29,-44.00,-0.97,-0.44,-10.12,-0.45,-1.00,-0.98,-23.19,-0.98,-0.81,-0.75,-1.04,-0.75,-0.80,-0.77,-3.16,-0.77,-1.00,-1.00,-48.25,-2.00,-1.00,-1.00,"-2,437.60",-16.00,-0.69,-0.68,-0.84,-0.69
25%,-0.07,-0.01,-0.09,-0.02,-0.08,-0.01,-0.11,-0.02,-0.07,-0.01,-0.09,-0.02,-0.07,-0.01,-0.10,-0.01,-0.07,-0.01,-0.08,-0.01,-0.06,-0.01,-0.07,-0.01,-0.05,-0.00,-0.06,-0.01,-0.07,-0.01,-0.09,-0.01,-0.05,-0.01,-0.08,-0.01,-0.08,-0.01,-0.12,-0.01
50%,-0.01,-0.00,-0.00,-0.00,-0.01,-0.00,-0.01,-0.00,-0.01,-0.00,-0.00,-0.00,-0.01,-0.00,-0.00,-0.00,-0.01,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.01,0.00
75%,0.07,0.01,0.09,0.01,0.07,0.01,0.10,0.01,0.07,0.01,0.09,0.01,0.08,0.00,0.10,0.01,0.06,0.01,0.08,0.01,0.06,0.01,0.08,0.02,0.06,0.01,0.08,0.

The next code cell filters down the regression dataset by filtering outliers.

Change the 'perc_limit' variable below only if your 25%/75% percentile values are greater than 3 in absolute terms (Use the Guideline comment). 

Else run as-is

Finally, compare the Count row to see how many rows have been filtered out and adjust accordingly

In [46]:
# Filter out extreme swings in non-forecasted sales i.e. greater than 300% change (likely due to some Promo effect)

# Select only indexes where each column value is between min & max
perc_limit = 1.5 # Guideline: Set to 1 + absolute value of (max of 25%/75% percentiles)

"""
# ------ APPROACH 1: REQUIRES PERFECT MATCHING. DOES NOT SCALE WELL IF  ATTRIBUTE HAS > 20 UNIQUE VALUES
# Only works if the numeric cols are the same scale e.g. Percentages
chosen_idx_list = []
for col in forecastdf_dummy_agg.select_dtypes('number').columns:
    chosen_idx_list.append(pd.DataFrame(forecastdf_dummy_agg[abs(forecastdf_dummy_agg[col]) <= perc_limit].index, index=forecastdf_dummy_agg[abs(forecastdf_dummy_agg[col]) <= perc_limit].index))
# Selects only the indices that match across all the columns
forecastdf_dummy_agg_filtered = forecastdf_dummy_agg.iloc[
    pd.concat([k for k in chosen_idx_list], axis=1, join='inner').iloc[:,0].index]
"""

# ------ APPROACH 2: REPLACE WITH NA WHICH REGRESSION MODEL WILL IMPUTE
forecastdf_dummy_agg_filtered = forecastdf_dummy_agg.copy()
for col in forecastdf_dummy_agg.select_dtypes('number').columns:
    forecastdf_dummy_agg_filtered.loc[abs(forecastdf_dummy_agg_filtered[col]) > perc_limit, col] = np.nan
    
# Drop rows where all values are NA
forecastdf_dummy_agg_filtered.dropna(axis=0, how='all')
#Interpolate NA rows based on time series
forecastdf_dummy_agg_filtered = forecastdf_dummy_agg_filtered.sort_values(by=['Retailer','ds']).set_index('ds').interpolate(method='time').reset_index().dropna()
forecastdf_dummy_agg_filtered.describe()

,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ENFANTS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ENFANTS,val_chg2_perc_SEG_DESC_ATTRIBUTE_ENFANTS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ENFANTS
count,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00,969.00
mean,0.00,0.00,0.00,0.00,-0.01,-0.00,-0.00,-0.01,-0.02,0.00,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.03,-0.01,-0.03,-0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.00,-0.00,0.00,-0.00,0.00,-0.01,0.01,-0.01,0.02,0.00,0.00,0.00,-0.00
std,0.12,0.07,0.16,0.08,0.19,0.09,0.26,0.10,0.20,0.11,0.26,0.12,0.17,0.10,0.25,0.12,0.21,0.14,0.27,0.16,0.18,0.07,0.23,0.09,0.16,0.09,0.19,0.11,0.20,0.12,0.25,0.14,0.20,0.14,0.27,0.15,0.16,0.07,0.22,0.09
min,-0.44,-0.33,-0.59,-0.32,-0.99,-1.00,-1.19,-1.01,-1.00,-1.00,-1.29,-0.98,-0.97,-0.44,-1.16,-0.45,-1.00,-0.98,-1.29,-0.98,-0.81,-0.75,-1.04,-0.75,-0.80,-0.77,-0.79,-0.77,-1.00,-1.00,-1.50,-1.01,-1.00,-1.00,-1.22,-1.02,-0.69,-0.68,-0.84,-0.69
25%,-0.07,-0.01,-0.09,-0.02,-0.08,-0.01,-0.11,-0.02,-0.07,-0.01,-0.09,-0.02,-0.07,-0.01,-0.10,-0.01,-0.07,-0.01,-0.08,-0.01,-0.06,-0.01,-0.07,-0.01,-0.05,-0.00,-0.06,-0.00,-0.07,-0.01,-0.09,-0.01,-0.05,-0.01,-0.08,-0.01,-0.08,-0.01,-0.13,-0.01
50%,-0.01,-0.00,-0.00,-0.00,-0.01,-0.00,-0.01,-0.00,-0.01,-0.00,-0.00,-0.00,-0.01,-0.00,-0.01,-0.00,-0.01,-0.00,-0.01,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.01,0.01,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.01,0.00
75%,0.06,0.01,0.08,0.01,0.07,0.01,0.09,0.01,0.06,0.01,0.08,0.01,0.07,0.00,0.09,0.01,0.05,0.01,0.07,0.01,0.06,0.01,0.08,0.02,0.06,0.01,0.08,0.02,0.06,0.01,0.

In [47]:
# Run as is
perc_rows_selected = forecastdf_dummy_agg_filtered['ds'].count() / forecastdf_dummy_agg['ds'].count()
print(f"Selected {perc_rows_selected*100: .2f}% of the dataset rows for regression \n")
print(f"as these have less than a {perc_limit*100: .2f}% swing in value vs. prior period")

Selected  99.69% of the dataset rows for regression 

as these have less than a  150.00% swing in value vs. prior period


In [48]:
# Run as is
# Create actual regression datasets
regression_df = forecastdf_dummy_agg_filtered.loc[:, 
                                     ['Retailer','ds'] + 
                                     forecastdf_dummy_agg_filtered.columns[
                                         forecastdf_dummy_agg_filtered.columns.str.contains('perc') & 
                                         # ensure line below has your attribute name
                                         forecastdf_dummy_agg_filtered.columns.str.contains(FOCUS_ATTRIBUTE_COLUMN)
                                     ].to_flat_index().to_list()].reset_index(drop=True)

# Split datasets
regression_df = regression_df.loc[:,~regression_df.columns.duplicated()] # Drop duplicate columns

## Categorical column transformations
# Encodes the categorical columns as strings so they can work with xgboost
regression_df['ds_month'] = regression_df['ds'].dt.month
regression_df['ds_year'] = regression_df['ds'].dt.year
# Note: LabelEncoder encodes a cardinal order e.g. 1>2>3 - Relevant for dates
lbl_month = LabelEncoder()
regression_df['ds_month'] = lbl_month.fit_transform(regression_df['ds_month'].astype(str))
lbl_year = LabelEncoder()
regression_df['ds_year'] = lbl_year.fit_transform(regression_df['ds_year'].astype(str))
# Onehot encode Market label & drops it from set
regression_df = pd.get_dummies(regression_df, columns=['Retailer'])  

# Create train & test dfs
regression_df = regression_df.drop('ds', axis=1)
regression_df_train = regression_df.sample(frac=0.8, random_state=42) # Selects 80% of dataset for main training
regression_df_test = regression_df.drop(regression_df_train.index) # Selects remainder of dataset for sight-unseen testing
regression_df_train.reset_index(drop=True, inplace=True)
regression_df_test.reset_index(drop=True, inplace=True)
regression_df_train

,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,...,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,val_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ENFANTS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_ENFANTS,val_chg2_perc_SEG_DESC_ATTRIBUTE_ENFANTS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_ENFANTS,ds_month,ds_year,Retailer_Auchan,Retailer_Auchan_SM,Retailer_CRF_HM,Retailer_CRF_Market,Retailer_Casino,Retailer_Cora,Retailer_Geant,Retailer_Monoprix,Retailer_U_enseigne
0,0.21,-0.00,0.30,-0.01,0.17,-0.00,0.30,-0.02,-0.07,-0.01,0.07,-0.01,0.01,-0.02,0.14,-0.03,0.03,-0.01,0.18,-0.00,0.01,-0.02,0.09,-0.01,-0.61,...,-0.58,-0.16,0.16,0.09,0.23,0.08,0.12,-0.00,0.24,-0.01,0.18,-0.02,0.28,-0.00,1,0,0,0,0,0,0,1,0,0,0
1,0.04,-0.14,0.02,-0.10,0.14,-0.05,-0.06,-0.04,0.01,-0.01,-0.02,-0.02,0.13,-0.00,0.01,-0.00,0.06,0.00,0.05,0.01,0.03,-0.11,-0.09,-0.11,0.06,...,0.01,0.00,0.05,-0.00,-0.00,-0.01,-1.00,-1.00,-0.84,-0.98,0.07,-0.00,0.04,0.04,2,0,0,0,1,0,0,0,0,0,0
2,-0.05,0.01,-0.10,-0.02,0.01,0.01,-0.08,0.01,0.03,0.02,-0.02,-0.01,-0.01,0.00,-0.07,-0.01,-0.00,0.00,-0.04,0.02,-0.03,0.01,-0.11,-0.00,-0.02,...,-0.03,0.06,-0.04,0.01,-0.10,-0.02,-0.02,0.01,-0.06,-0.00,-0.06,0.01,-0.14,0.00,6,1,0,0,0,1,0,0,0,0,0
3,-0.08,0.03,-0.15,-0.05,-0.23,-0.00,-0.24,-0.02,-0.08,-0.02,-0.08,-0.01,-0.10,-0.00,-0.11,-0.01,-0.10,-0.00,-0.11,-0.01,-0.06,-0.00,-0.32,-0.02,-0.03,...,-0.03,-0.02,-0.00,-0.01,-0.09,-0.02,-0.03,-0.01,-0.07,-0.02,0.00,0.01,-0.06,0.01,6,1,0,0,0,0,1,0,0,0,0
4,0.13,0.01,0.20,0.02,0.07,0.02,0.23,0.06,0.17,-0.01,0.14,-0.01,0.10,0.05,0.25,0.05,-0.66,-0.00,-0.88,0.00,0.04,-0.00,0.17,0.00,0.00,...,0.13,0.05,0.08,0.05,0.13,0.07,-0.73,0.02,-0.71,0.04,0.16,0.00,0.95,0.25,8,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,0.04,-0.03,0.07,-0.02,-0.02,-0.02,-0.00,-0.0

We will predict the transferable demand for each of these columns below in turn

In [49]:
# Run as is
prediction_cols = regression_df_train.columns[
    (regression_df_train.columns.str.contains('val_chg1')) & 
    (regression_df_train.columns.str.contains(FOCUS_ATTRIBUTE_COLUMN))].to_flat_index().to_list()
regression_df_train[prediction_cols].describe()

,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - AO MARQUES,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - COLGATE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - ORAL B,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - SIGNAL,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - NON PHARMA - VADMECUM,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - ELMEX,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - PARADONTAX,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SANOGYL,val_chg1_perc_SEG_DESC_ATTRIBUTE_ADULTE - PHARMA - SENSODYNE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ENFANTS
count,775.00,775.00,775.00,775.00,775.00,775.00,775.00,775.00,775.00,775.00
mean,-0.00,-0.01,-0.02,-0.01,-0.03,0.01,0.01,-0.00,-0.01,0.00
std,0.12,0.20,0.20,0.17,0.21,0.18,0.16,0.20,0.21,0.16
min,-0.44,-0.99,-1.00,-0.97,-1.00,-0.81,-0.80,-1.00,-1.00,-0.69
25%,-0.07,-0.08,-0.07,-0.07,-0.07,-0.06,-0.05,-0.06,-0.05,-0.08
50%,-0.01,-0.01,-0.01,-0.01,-0.01,-0.00,0.01,-0.00,-0.00,-0.00
75%,0.06,0.07,0.06,0.07,0.05,0.06,0.06,0.06,0.05,0.07
max,0.61,1.48,1.46,1.30,1.05,1.27,1.38,1.50,1.38,1.41


In [50]:
# Run as is
## Bayes Robust Approach
final_coefs_dict = {} # defaultdict(pd.DataFrame)
final_scores_dict = {}

for col in prediction_cols:
    my_param_grid = {
        'tol':[1e-3, 1e-4],
        'alpha_1':[1e-6, 1e-8, 1e-4],
        'lambda_init':[1, 0.5, 10]    
    }
    reg_train_pipe = Pipeline([
        ('transformer', QuantileTransformer(output_distribution='normal')),
        ('model', GridSearchCV(BayesianRidge(n_iter=500, fit_intercept=True), 
                                    param_grid=my_param_grid, scoring='r2', n_jobs=-1))
    ])
    clf = reg_train_pipe.fit(regression_df_train.drop(col, axis=1), regression_df_train.loc[:,col])
    myscore = clf.score(regression_df_test.drop(col, axis=1), regression_df_test.loc[:,col])
    final_scores_dict[col] = pd.DataFrame(pd.Series(myscore, index=[col],name='Score'))
    temp_coefs_df = pd.DataFrame(list(zip(regression_df_train.drop(col, axis=1).columns, clf['model'].best_estimator_.coef_)), columns=['Feature','Coefficient'])
    # Select only the dist_chg numbers
    final_coefs_dict[col] = temp_coefs_df.loc[
        temp_coefs_df['Feature'].str.contains(FOCUS_ATTRIBUTE_COLUMN) & 
        temp_coefs_df['Feature'].str.contains('dist_chg1')].copy()
 

C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\sklearn\preprocessing\_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (775). n_quantiles is set to n_samples.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\sklearn\preprocessing\_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (775). n_quantiles is set to n_samples.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\sklearn\preprocessing\_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (775). n_quantiles is set to n_samples.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\sklearn\preprocessing\_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (775). n_quantiles is set to n_samples.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\datascience\lib\site-packages\sklearn\preprocessin

In [51]:
final_coefs = pd.DataFrame()
final_scores = pd.DataFrame()

for k in final_coefs_dict.keys():
    final_coefs_dict[k].loc[:,'TO_ATTRIBUTE'] = k # Set the key as a column in the dataframe
    final_coefs = final_coefs.append(final_coefs_dict[k].copy(), ignore_index=True)
    
    final_scores_dict[k].loc[:,'MODEL_NAME'] = k # Set the key as a column in the dataframe
    final_scores = final_scores.append(final_scores_dict[k])

# Create final accuracy scores
final_scores['MODEL_NAME'] = final_scores['MODEL_NAME'].str.replace("val_chg1_perc_","").str.replace(FOCUS_ATTRIBUTE_COLUMN+"_","")
final_scores_bayes = final_scores.copy()

# Format & adjust coefs
final_coefs.rename(columns={'Feature':'FROM_ATTRIBUTE'}, inplace=True)

final_coefs['TO_ATTRIBUTE'] = final_coefs['TO_ATTRIBUTE'].str.replace("val_chg1_perc_","").str.replace(FOCUS_ATTRIBUTE_COLUMN+"_","")
final_coefs['FROM_ATTRIBUTE'] = final_coefs['FROM_ATTRIBUTE'].str.replace("dist_chg1_perc_","").str.replace(FOCUS_ATTRIBUTE_COLUMN+"_",'')

"""
# Keep self interaction by ensuring it is negative i.e. substitute
#final_coefs.loc[(final_coefs['TO_ATTRIBUTE'] == final_coefs['FROM_ATTRIBUTE']) & 
#                (final_coefs['Coefficient']>0), 'Coefficient'] = final_coefs.loc[(final_coefs['TO_ATTRIBUTE'] == final_coefs['FROM_ATTRIBUTE']) & 
#                                                                                 (final_coefs['Coefficient']>0), 'Coefficient'] * -1
# For ISD, remove self elasts
final_coefs = final_coefs.loc[~(final_coefs['TO_ATTRIBUTE'] == final_coefs['FROM_ATTRIBUTE'])]
                              
# Zero out positive coefficients i.e. Increase Dist -> Increase value: Implies complements vs. substitutes
final_coefs.loc[final_coefs['Coefficient'] > 0, 'Coefficient'] = 0

# Adjust final coefficients for export

# Rebase % transfers to 100%
final_coefs_adjusted_bayes = final_coefs.merge(
    final_coefs[['TO_ATTRIBUTE','Coefficient']].groupby(['TO_ATTRIBUTE']).sum(), 
    on=['TO_ATTRIBUTE'], suffixes=("","_sum")
)

final_coefs_adjusted_bayes.loc[:, 'Percent_Transfer'] = final_coefs_adjusted_bayes['Coefficient'] / final_coefs_adjusted_bayes['Coefficient_sum']
final_coefs_adjusted_bayes[['TO_ATTRIBUTE','FROM_ATTRIBUTE','Percent_Transfer']].sort_values(['TO_ATTRIBUTE','Percent_Transfer'], ascending=False)
"""
# For ISD, remove self elasts
final_coefs = final_coefs.loc[~(final_coefs['TO_ATTRIBUTE'] == final_coefs['FROM_ATTRIBUTE'])]

# Invert the scale so the substitutes (-ve coeff) are now positives
final_coefs.loc[:,'Coefficient'] = final_coefs.loc[:,'Coefficient'] *-1

# Finalise Percent Transfer by abs-value shift to account for -ve coefficients
# Note: Complements i.e. highly negative coeffs (post inversion) will be close to zero and thus regularised naturally
final_coefs = final_coefs.merge(
    final_coefs[['TO_ATTRIBUTE','Coefficient']].groupby(['TO_ATTRIBUTE']).min(), 
    on=['TO_ATTRIBUTE'], suffixes=("","_min")
)

final_coefs['Coefficient_minabs'] = abs(final_coefs['Coefficient_min'])
final_coefs['Coefficient_pos_shift'] = final_coefs['Coefficient'] + final_coefs['Coefficient_minabs']

final_coefs_adjusted_bayes = final_coefs.merge(
    final_coefs[['TO_ATTRIBUTE','Coefficient_pos_shift']].groupby(['TO_ATTRIBUTE']).sum(), 
    on=['TO_ATTRIBUTE'], suffixes=("","_sum")
)

final_coefs_adjusted_bayes.loc[:, 'Percent_Transfer'] = final_coefs_adjusted_bayes['Coefficient_pos_shift'] / final_coefs_adjusted_bayes['Coefficient_pos_shift_sum']
final_coefs_adjusted_bayes = final_coefs_adjusted_bayes[['TO_ATTRIBUTE','FROM_ATTRIBUTE','Percent_Transfer']].sort_values(['TO_ATTRIBUTE','Percent_Transfer'], ascending=False)


final_coefs_adjusted_bayes

C:\Users\ibrahimad\AppData\Local\Temp\ipykernel_11984\3936643298.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_coefs = final_coefs.append(final_coefs_dict[k].copy(), ignore_index=True)
C:\Users\ibrahimad\AppData\Local\Temp\ipykernel_11984\3936643298.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_scores = final_scores.append(final_scores_dict[k])
C:\Users\ibrahimad\AppData\Local\Temp\ipykernel_11984\3936643298.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_coefs = final_coefs.append(final_coefs_dict[k].copy(), ignore_index=True)
C:\Users\ibrahimad\AppData\Local\Temp\ipykernel_11984\3936643298.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futu

,TO_ATTRIBUTE,FROM_ATTRIBUTE,Percent_Transfer
82,ENFANTS,ADULTE - NON PHARMA - COLGATE,0.24
83,ENFANTS,ADULTE - NON PHARMA - ORAL B,0.17
88,ENFANTS,ADULTE - PHARMA - SANOGYL,0.17
87,ENFANTS,ADULTE - PHARMA - PARADONTAX,0.15
81,ENFANTS,ADULTE - NON PHARMA - AO MARQUES,0.13
...,...,...,...
2,ADULTE - NON PHARMA - AO MARQUES,ADULTE - NON PHARMA - SIGNAL,0.11
7,ADULTE - NON PHARMA - AO MARQUES,ADULTE - PHARMA - SENSODYNE,0.10
3,ADULTE - NON PHARMA - AO MARQUES,ADULTE - NON PHARMA - VADMECUM,0.10
5,ADULTE - NON PHARMA - AO MARQUES,ADULTE - PHARMA - PARADONTAX,0.08


In [52]:
final_coefs_adjusted_bayes[['TO_ATTRIBUTE','FROM_ATTRIBUTE','Percent_Transfer']].sort_values(['TO_ATTRIBUTE','Percent_Transfer'], ascending=False).to_excel('./'+ANALYSIS_CATEGORY+' ATTR_SPECIFIC_BAYES_'+FOCUS_ATTRIBUTE_COLUMN+'_COEFS.xlsx', index=False)
final_scores_bayes.sort_values('MODEL_NAME', ascending=True).to_excel('./'+ANALYSIS_CATEGORY+' ATTR_SPECIFIC_BAYES_'+FOCUS_ATTRIBUTE_COLUMN+'_SCORES.xlsx', index=False)

YOU'RE DONE!!!

Percent Transfers have been outputted to file that starts with your Category Name & Attribute of Focus in this directory